# FRESCO Database Access and Analytics Notebook

## Overview

The FRESCO project is collecting, curating, and presenting public failure data of large-scale computing systems, in a repository called FRESCO. Our initial sources are Purdue, U of Illinois at Urbana-Champaign, and U of Texas at Austin. The data sets comprise static and dynamic information about system usage and the workloads, and failure information, for both planned and unplanned outages. The purpose of this notebook is to provide public access to this data.

This notebook contains three cells:

### 1. Data Filtering
Begin by selecting a specific datetime window for either (or both) databases. Customize your dataset further with various filters.

### 2. Data Analysis Options
Explore a range of analysis options and select the ones that align with your requirements.

### 3. Data Analysis and Visualizations
Execute the chosen analysis on the filtered dataset and visualize the outcomes.

## Step-by-Step Instructions

- **Cell 1:**
1. Run the first cell by clicking inside of it and using Ctrl+Enter.
2. Define your query's temporal boundaries and click the "Validate Dates" button.
3. If desired, use the filtering options to further narrow your query.
4. Select "Execute Query"

- **Cell 2:** Configure your statistical analyses:
1. Run the cell.
2. Choose the statistics to be calculated (e.g., Mean, Median).
3. If "Ratio of Data Outside Threshold" was selected, provide the threshold.
4. Select an interval type (Count or Time). If "Time" is chosen, define the time unit and interval count.
5. Provide the interval value.

- **Cell 3:** Run this cell to run the analyses selected in cell 2.

## Database Table Information

### Host Table
- **jid**: Unique job identifier
- **host**: Origin node of the data point
- **event**: Resource usage metric type
- **value**: Numeric value of the metric
- **unit**: Measurement unit of the metric
- **time**: Timestamp of the data point


**Event Column Metrics:**
- **cpuuser:** CPU user mode average percentage.
- **block:** Data transfer rate to/from block devices.
- **memused:** OS's total physical memory usage.
- **memused_minus_diskcache:** Physical memory usage excluding caches.
- **gpu_usage:** GPU active time average percentage (only for GPU jobs).
- **nfs:** Data transfer rate over NFS mounts.

### Job Table
- **account**: Account or project name
- **jid**: Unique job identifier
- **ncores**: Total cores assigned to the job
- **ngpus**: Total GPUs assigned to the job
- **nhosts**: Number of nodes assigned to the job
- **timelimit**: Requested job duration (in seconds)
- **queue**: Job submission queue name
- **end_time**: Job end time
- **start_time**: Job start time
- **submit_time**: Job submission time
- **username**: Job owner's name
- **exitcode**: Job's exit status
- **host_list**: List of nodes the job ran on
- **jobname**: Job's name

In [1]:
from classes.notebook_utilities import NotebookUtilities

nu = NotebookUtilities()

GridBox(children=(VBox(children=(HTML(value='<h1>Query the Host Data Table</h1>'), HTML(value='<h4>Select star…

In [ ]:
nu.display_statistics_widgets()

In [ ]:
%matplotlib inline
nu.display_plots()